In [ ]:
!pip install fasttext
!pip install simpletransformers

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.te.zip

--2023-06-14 06:30:57--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.te.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.224.2.6, 13.224.2.88, 13.224.2.21, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.224.2.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2990021886 (2.8G) [application/zip]
Saving to: ‘wiki.te.zip’

wiki.te.zip         100%[===================>]   2.78G  35.7MB/s    in 78s     

2023-06-14 06:32:15 (36.4 MB/s) - ‘wiki.te.zip’ saved [2990021886/2990021886]



In [ ]:
!unzip /content/wiki.te.zip

Archive:  /content/wiki.te.zip
  inflating: wiki.te.vec             
  inflating: wiki.te.bin             


In [ ]:
import fasttext
import fasttext.util
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
def f1_score(precision, recall):
    return 2 * ((precision * recall) / (precision + recall))

In [ ]:
ft = fasttext.load_model('wiki.te.bin')

In [ ]:
mr_test = pd.read_csv('/content/CSE 256 Results - Telugu - 5 Class(Test).csv')
mr_train = pd.read_csv('/content/CSE 256 Results - Telugu - 5 Class(Train).csv')

In [ ]:
mr_train

,Sentence,Label
0,స్టార్‌ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటి...,entertainment
1,"నిర్మాణ సంస్థ: డిజిక్వెస్ట్ ఇండియా లిమిటెడ్‌,...",entertainment
2,గ‌త నెల 28న విడుద‌లైన బాహుబ‌లి-2 సినిమా సంచ‌ల‌...,entertainment
3,"ఈసారి చిరు, బాలయ్య సినిమాలతో సంక్రాంతి అత్యంత ...",entertainment
4,కన్నడలో హిట్‌గా నిలిచిన 'రాజధాని' చిత్రాన్ని త...,entertainment
...,...,...
95,\nరాయిమీదా రాయిగొట్టీరాజుకున్నా నిప్పుబట్టివంట...,editorial
96,ముఖ్యమంత్రి మెహబూబా ముఫ్తీని జైరా కలుసుకున్న ...,editorial
97,రానురాను దేశం హింసాత్మకంగా మారుతోంది. జాతీయ నే...,editorial
98,ప్రభుత్వాలు తమను కరుణించే స్థితిలో లేవని ఆదివా...,editorial


In [ ]:
mr_test['Label'].value_counts()

sports           21
entertainment    20
nation           20
editorial        20
business         19
Name: Label, dtype: int64

In [ ]:
X = mr_train['Sentence'].tolist()
y = mr_train['Label'].tolist()

In [ ]:
# word = "अवलादी"
# print("Embedding Shape is {}".format(ft.get_word_vector(word).shape))
# print("Nearest Neighbors to {} are:".format(word))
# ft.get_nearest_neighbors(word)[0][1]

### Augmentation - Synonym

In [ ]:
def EDA_Synonym_Replacement(X, y, n_words, n_times):
  delimiter = ' '
  augmented_X = list()
  augmented_y = list()

  for i in range(len(X)):
    temp_n_words = n_words
    for j in range(n_times):
      data = X[i].split(" ")
      if(len(data) < n_words):
        temp_n_words = int(len(data)/2)
      indexes = random.sample(range(1, len(data)+1), temp_n_words)
      new_data = list()
      for k in range(len(data)):
        if k in indexes:
          new_data.append(ft.get_nearest_neighbors(data[k])[0][1])
        else:
          new_data.append(data[k])
      augmented_X.append(delimiter.join(new_data))
      augmented_y.append(y[i])
    augmented_X.append(X[i])
    augmented_y.append(y[i])
  return augmented_X, augmented_y

In [ ]:
augmented_X_1, augmented_y_1 = EDA_Synonym_Replacement(X, y, 3, 1)

In [ ]:
unique_values = list(set(augmented_y_1))
print(unique_values)

['business', 'nation', 'editorial', 'sports', 'entertainment']


In [ ]:
len(augmented_X_1) == len(augmented_y_1)

True

In [ ]:
df_augmentation_1 = pd.DataFrame({'text': augmented_X_1, 'labels': augmented_y_1})
df_augmentation_1 = df_augmentation_1.sample(frac=1).reset_index(drop=True)

In [ ]:
rows_with_nan = df_augmentation_1[df_augmentation_1.isnull().any(axis=1)]

print(rows_with_nan)

Empty DataFrame
Columns: [text, labels]
Index: []


In [ ]:
df_augmentation_1 = df_augmentation_1.dropna()

In [ ]:
rows_with_nan = df_augmentation_1[df_augmentation_1.isnull().any(axis=1)]

print(rows_with_nan)

Empty DataFrame
Columns: [text, labels]
Index: []


In [ ]:
test_data = mr_test.rename(columns={'Sentence': 'text', 'Label': 'labels'})

In [ ]:
le = LabelEncoder()

In [ ]:
mr_train['label_encoded'] = le.fit_transform(mr_train['Label'])

In [ ]:
df_augmentation_1

,text,labels
0,"\nయూనివర్సిటీల పరిరక్షణకై ప్రగతిశీల, ప్రజాస్వా...",editorial
1,"మన దేశంలో బ్యాంకింగ్‌, బీమా, ఎంఎఫ్‌ రంగాల్లో న...",business
2,కాకినాడ (ఆంధ్రజ్యోతి): తూర్పు గోదావరి జిల్లా క...,business
3,"న్యూఢిల్లీ : జాతీయకాంగ్రెస్ సీనియర్ నేత, కేంద్...",nation
4,కటక్: ఇంగ్లండ్‌తో జరుగుతున్న సెకండ్ వన్డే మ్యా...,sports
...,...,...
195,స్టార్‌ హీరోగా వరుస సినిమాలకీ బిజీగా ఉన్నప్పటి...,entertainment
196,అబుధాబి: శ్రీలంక టాప్ క్రికెటర్ చమర కపుగెదర పె...,sports
197,బాలీవుడ్‌ నటుడు టామ్‌ ఆల్టర్‌(67) కాన్సర్‌తో బ...,entertainment
198,కన్సాస్ : విద్వేషపూరిత దాడుల నేపథ్యంలో ప్రవాస ...,nation


In [ ]:
df_augmentation_1['label_encoded'] = le.fit_transform(df_augmentation_1['labels'])
test_data['label_encoded'] = le.fit_transform(test_data['labels'])

In [ ]:
df_augmentation_1 = df_augmentation_1.drop('labels', axis=1)
test_data = test_data.drop('labels', axis=1)

df_augmentation_1 = df_augmentation_1.rename(columns={'text': 'text', 'label_encoded': 'labels'})
test_data = test_data.rename(columns={'text': 'text', 'label_encoded': 'labels'})

In [ ]:
test_data

,text,labels
0,బిజినెస్‌మ్యాన్ ముఖేష్ అంబానీ ఏర్పాటు చేసిన ఒక...,2
1,పుట్టినరోజు సందర్భంగా అభిమానుల కోసం రెండు అద్భ...,2
2,నటసింహ బాలకృష్ణ 102వ సినిమా షూటింగ్ శరవేగంగా జ...,2
3,సిర్సా: గతంలో టాలీవుడ్ అగ్రహీరోలతో విజయవంతమైన ...,2
4,హైదరాబాద్‌: సినీ తారలను డ్రగ్స్‌ కేసు వెంటాడుత...,2
...,...,...
95,హైదరాబాద్‌ మహానగరం నేడు రెండు ప్రధాన ఘట్టాలకు ...,1
96,‘తెలుగువాడు కాదు తెగడకున్నతెలుగువాడు కాదు తెగు...,1
97,ఆర్‌సీఈపీ ఒప్పందం అమల్లోకొస్తే ఇప్పటివరకూ పాక్...,1
98,మా నాన్న 20 ఏళ్ల క్రితం చెప్పినట్టు పోరాటం జరగ...,1


In [ ]:
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir = True, max_seq_length=512)

# Create a ClassificationModel
model_roberta = ClassificationModel("bert", "bert-base-multilingual-cased", args=model_args, num_labels=5)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

In [ ]:
df_augmentation_1['labels'].value_counts()

1    40
0    40
3    40
4    40
2    40
Name: labels, dtype: int64

In [ ]:
# test_data.columns

In [ ]:
# test_data  = test_data.drop(['Unnamed: 2'], axis=1)

In [ ]:
# Train the model
model_roberta.train_model(df_augmentation_1)

# Evaluate the model
result, model_outputs, wrong_predictions = model_roberta.eval_model(test_data)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/25 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/25 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
predictions, raw_outputs = model_roberta.predict(test_data['text'].tolist())
true_labels = test_data['labels'].tolist()

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
precision_macro = precision_score(true_labels, predictions, average = 'macro')
recall_macro = recall_score(true_labels, predictions, average = 'macro')

precision_micro = precision_score(true_labels, predictions, average = 'micro')
recall_micro = recall_score(true_labels, predictions, average = 'micro')


accuracy = accuracy_score(true_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision_macro)
print("Recall:", recall_macro)
f1 = f1_score(precision_macro, recall_macro)
print("F1 score Macro Aug:", f1)
f1 = f1_score(precision_micro, recall_micro)
print("F1 score Micro Aug:", f1)

Accuracy: 0.72
Precision: 0.7540039754558812
Recall: 0.7198496240601504
F1 score Macro Aug: 0.7365310617689624
F1 score Micro Aug: 0.72


In [ ]:
# Precision: 0.6888888888888889
# Recall: 0.6862745098039216
# Accuracy: 0.6862745098039216
# F1 score Aug: 0.6875792141951839

### Augmentation - Baseline

In [ ]:
mr_train['label_encoded'].value_counts()

1    51
0    50
2     1
Name: label_encoded, dtype: int64

In [ ]:
mr_train = mr_train.drop(mr_train[mr_train['label_encoded'] == 2].index)

In [ ]:
mr_train['label_encoded'].value_counts()

1    51
0    50
Name: label_encoded, dtype: int64

In [ ]:
mr_train.head(5)

,Sentence,Label \nHate or Not,label_encoded
0,@BhatkhalkarA तुझी लायकी काय तू बोलतो कोणा बदल...,HOF,0
1,हिंदू धर्माबद्दल अपशब्द उगारणारा मोकाट आन उस्म...,HOF,0
2,"@abpmajhatv खांडक्या,विघानसभेत अजितदादा काय बो...",HOF,0
3,@ashish_jadhao @AUThackeray सरकार मधला बावळट स...,HOF,0
4,"ब्राह्मणवाद इतका वाईट, निच, आणि कपटी आहे की स्...",HOF,0


In [ ]:
mr_train = mr_train.drop('Label \nHate or Not', axis=1)

mr_train = mr_train.rename(columns={'Sentence': 'text', 'label_encoded': 'labels'})

In [ ]:
mr_train.head(5)

,text,labels
0,@BhatkhalkarA तुझी लायकी काय तू बोलतो कोणा बदल...,0
1,हिंदू धर्माबद्दल अपशब्द उगारणारा मोकाट आन उस्म...,0
2,"@abpmajhatv खांडक्या,विघानसभेत अजितदादा काय बो...",0
3,@ashish_jadhao @AUThackeray सरकार मधला बावळट स...,0
4,"ब्राह्मणवाद इतका वाईट, निच, आणि कपटी आहे की स्...",0


In [ ]:
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir = True, max_seq_length=512)

# Create a ClassificationModel
model_roberta = ClassificationModel("bert", "bert-base-multilingual-cased", args=model_args, num_labels=2)


# Train the model
model_roberta.train_model(mr_train)

# Evaluate the model
result, model_outputs, wrong_predictions = model_roberta.eval_model(test_data)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
predictions, raw_outputs = model_roberta.predict(test_data['text'].tolist())
true_labels = test_data['labels'].tolist()

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
precision = precision_score(true_labels, predictions, average = 'macro')
recall = recall_score(true_labels, predictions, average = 'macro')
accuracy = accuracy_score(true_labels, predictions)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
f1 = f1_score(precision, recall)
print("F1 score Aug:", f1)

Precision: 0.7049107142857143
Recall: 0.6764705882352942
Accuracy: 0.6764705882352942
F1 score Aug: 0.6903978860521263


### Augmentation - Random Insertion

In [ ]:
!pip install advertools

In [ ]:
import advertools as adv
adv.stopwords.keys()

dict_keys(['arabic', 'azerbaijani', 'bengali', 'catalan', 'chinese', 'croatian', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hindi', 'hungarian', 'indonesian', 'irish', 'italian', 'japanese', 'kazakh', 'nepali', 'norwegian', 'persian', 'polish', 'portuguese', 'romanian', 'russian', 'sinhala', 'spanish', 'swedish', 'tagalog', 'tamil', 'tatar', 'telugu', 'thai', 'turkish', 'ukrainian', 'urdu', 'vietnamese'])

In [ ]:
stop_words = sorted(adv.stopwords['telugu'])

In [ ]:
# https://www.kaggle.com/datasets/rtatman/stopword-lists-for-19-languages?select=marathiST.txt
# https://github.com/stopwords-iso/stopwords-mr/blob/master/stopwords-mr.txt

In [ ]:
# !wget https://raw.githubusercontent.com/stopwords-iso/stopwords-mr/master/stopwords-mr.txt

--2023-06-14 04:51:03--  https://raw.githubusercontent.com/stopwords-iso/stopwords-mr/master/stopwords-mr.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1265 (1.2K) [text/plain]
Saving to: ‘stopwords-mr.txt’

stopwords-mr.txt    100%[===================>]   1.24K  --.-KB/s    in 0s      

2023-06-14 04:51:04 (53.5 MB/s) - ‘stopwords-mr.txt’ saved [1265/1265]



In [ ]:
# stop_words_1 = list()
# stop_words_2 = list()

# with open("marathiST.txt", "r") as file:
#     stop_words_1 = file.read().split()
# file.close()


# with open("stopwords-mr.txt", "r") as file:
#     stop_words_2 = file.read().split()
# file.close()


In [ ]:
# stop_words = list(set(stop_words_1).union(set(stop_words_2)))

In [ ]:
def EDA_Random_Insertion(X, y, n_times, stop_words):
  delimiter = ' '
  augmented_X = list()
  augmented_y = list()

  for i in range(len(X)):
    for j in range(n_times):
      data = X[i].split(" ")
      indexes_temp = list()
      for k in range(len(data)):
        if data[k] not in stop_words:
          indexes_temp.append(k)
      index = random.choice(indexes_temp)
      new_word = ft.get_nearest_neighbors(data[index])[0][1]
      insert_index =  random.sample(range(1, len(data)+1), 1)
      data.insert(int(insert_index[0]), new_word)
      augmented_X.append(delimiter.join(data))
      augmented_y.append(y[i])
    augmented_X.append(X[i])
    augmented_y.append(y[i])
  return augmented_X, augmented_y

In [ ]:
augmented_X_2, augmented_y_2 = EDA_Random_Insertion(X, y, 1, stop_words)

In [ ]:
unique_values = list(set(augmented_y_2))
print(unique_values)

['business', 'nation', 'editorial', 'sports', 'entertainment']


In [ ]:
len(augmented_X_2) == len(augmented_y_2)

True

In [ ]:
df_augmentation_2 = pd.DataFrame({'text': augmented_X_2, 'labels': augmented_y_2})
df_augmentation_2 = df_augmentation_2.sample(frac=1).reset_index(drop=True)

In [ ]:
rows_with_nan = df_augmentation_2[df_augmentation_2.isnull().any(axis=1)]

print(rows_with_nan)

Empty DataFrame
Columns: [text, labels]
Index: []


In [ ]:
df_augmentation_2 = df_augmentation_2.dropna()

In [ ]:
rows_with_nan = df_augmentation_2[df_augmentation_2.isnull().any(axis=1)]

print(rows_with_nan)

Empty DataFrame
Columns: [text, labels]
Index: []


In [ ]:
# test_data = mr_test.rename(columns={'Sentence': 'text', 'Label \nHate or Not': 'labels'})

In [ ]:
# le = LabelEncoder()

In [ ]:
# mr_train['label_encoded'] = le.fit_transform(mr_train['Label \nHate or Not'])

In [ ]:
df_augmentation_2['label_encoded'] = le.fit_transform(df_augmentation_2['labels'])
# test_data['label_encoded'] = le.fit_transform(test_data['labels'])

In [ ]:
df_augmentation_2 = df_augmentation_2.drop('labels', axis=1)
# test_data = test_data.drop('labels', axis=1)

df_augmentation_2 = df_augmentation_2.rename(columns={'text': 'text', 'label_encoded': 'labels'})
# test_data = test_data.rename(columns={'text': 'text', 'label_encoded': 'labels'})

In [ ]:
test_data

,text,labels
0,బిజినెస్‌మ్యాన్ ముఖేష్ అంబానీ ఏర్పాటు చేసిన ఒక...,2
1,పుట్టినరోజు సందర్భంగా అభిమానుల కోసం రెండు అద్భ...,2
2,నటసింహ బాలకృష్ణ 102వ సినిమా షూటింగ్ శరవేగంగా జ...,2
3,సిర్సా: గతంలో టాలీవుడ్ అగ్రహీరోలతో విజయవంతమైన ...,2
4,హైదరాబాద్‌: సినీ తారలను డ్రగ్స్‌ కేసు వెంటాడుత...,2
...,...,...
95,హైదరాబాద్‌ మహానగరం నేడు రెండు ప్రధాన ఘట్టాలకు ...,1
96,‘తెలుగువాడు కాదు తెగడకున్నతెలుగువాడు కాదు తెగు...,1
97,ఆర్‌సీఈపీ ఒప్పందం అమల్లోకొస్తే ఇప్పటివరకూ పాక్...,1
98,మా నాన్న 20 ఏళ్ల క్రితం చెప్పినట్టు పోరాటం జరగ...,1


In [ ]:
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir = True, max_seq_length=512)

# Create a ClassificationModel
model_roberta = ClassificationModel("bert", "bert-base-multilingual-cased", args=model_args, num_labels=5)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

In [ ]:
df_augmentation_2['labels'].value_counts()

3    40
4    40
0    40
2    40
1    40
Name: labels, dtype: int64

In [ ]:
# Train the model
model_roberta.train_model(df_augmentation_2)

# Evaluate the model
result, model_outputs, wrong_predictions = model_roberta.eval_model(test_data)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/200 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_512_5_2


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/25 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_512_5_2


Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.6531874420573269, 'eval_loss': 0.7606541560246394}


In [ ]:
predictions, raw_outputs = model_roberta.predict(test_data['text'].tolist())
true_labels = test_data['labels'].tolist()

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
precision_macro = precision_score(true_labels, predictions, average = 'macro')
recall_macro = recall_score(true_labels, predictions, average = 'macro')

precision_micro = precision_score(true_labels, predictions, average = 'micro')
recall_micro = recall_score(true_labels, predictions, average = 'micro')


accuracy = accuracy_score(true_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision_macro)
print("Recall:", recall_macro)
f1 = f1_score(precision_macro, recall_macro)
print("F1 score Macro Aug:", f1)
f1 = f1_score(precision_micro, recall_micro)
print("F1 score Micro Aug:", f1)

Accuracy: 0.72
Precision: 0.7181961758048715
Recall: 0.7183208020050125
F1 score Macro Aug: 0.7182584834989182
F1 score Micro Aug: 0.72
